<a href="https://colab.research.google.com/github/raciel88p/Structured-data-python/blob/main/Copia_de_Articulos_IA_WP_Alvaro_Pe%C3%B1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Script generado por Álvaro Peña @isocialwebseo en twitter. Tienes libertad para modificar, mejorar y compartir parte o la totalidad del contenido de este script. 
# Eso sí, haz un uso responsable del mismo.

#Instalamos las dependencias necesarias
!pip install python-wordpress-xmlrpc
!pip install openai pandas requests

In [ ]:
# Subimos el archivo CSV con nombre de archivo: articulos.csv que contiene las keywords extraidas de Ahrefs, semrush, GSC... Tienen que estar en una única columna y la primera casillas llamarse title
from google.colab import files
uploaded = files.upload()

import openai
import re
import base64
import requests
import pandas as pd
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost

# Leemos el archivo CSV que subimos al entorno de Colab
df = pd.read_csv('articulos.csv')

#RELLENAR: Añade aquí tu API de OpenAI
openai.api_key = "API-OPENAI"
# wp credentials

#RELLENAR: Añade aquí tus credenciales de Wordpress
user = "USUARIO"
password = "CONTRASEÑAS"
url = "https://DOMINIO/wp-login.php"


for titulo in df['title']:
  try:
    # -------------------------

    input_titulo = "Genera un título llamativo para un artículo con la siguiente cadena: " + str(titulo)

    resp_titulo = openai.Completion.create(
        model="text-davinci-003",
        prompt=input_titulo,
        max_tokens=100,
        temperature=0.7,
    )

    resp_titulo = (resp_titulo.choices[0].text) 
    resp_titulo = resp_titulo.replace('"', '')
    print("Titulo: ", resp_titulo)

    input_contenido = "Genera un artículo completos de mínimo 700 palabras y máximo 1500 con el título: " + resp_titulo + " interesante para el usuario, con subtítulos en HTML con H2, acaba todas las frases aunque el texto se exceda del máximo. En este artículo se deben responder las principales dudas que tienen los usuarios en Google sobre este tema, responder sus preguntas frecuentes."

    resp_contenido = openai.Completion.create(
        model="text-davinci-003",
        prompt=input_contenido,
        max_tokens=1500,
        temperature=0.7,
    )

    summary = (resp_contenido.choices[0].text)
    summary = summary.replace('"', '')

    # RELLENAR: Vuelve a añadir tu usuario de wordpress y en el password la clave que se genera en tu usuario de WP, desde la sección perfil en "Contraseñas de aplicación" pones el nombre que quieras y pegas la contraseña que te genera
    login = 'USUARIO'
    password = 'CONTRASEÑA'

    # RELLENAR: Establecer la URL de la API de WordPress y los encabezados de autorización
    url = 'https://DOMINIO/wp-json/wp/v2/posts'
    headers = {
        'Authorization': 'Basic ' + base64.b64encode(f"{login}:{password}".encode()).decode()
    }

    # Establecer el cuerpo de la solicitud con el título y el estado del post
    data = {
        'title': resp_titulo,
        'content': summary,
        'status': 'publish'
    }

    # Realizar la solicitud POST
    response = requests.post(url, headers=headers, json=data)

    # Verificar el estado de la respuesta
    if response.status_code == 201:
        print('Post creado correctamente')
    else:
        print('Error al crear el post')

  except Exception:
    print("Error general")